In [ ]:
##Estimación de pobreza de telecomunicaciones de acuerdo con Evalúa CDMX
#Se borra todo lo que se encuentra en el entorno

rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr,showtext,gt)

#Fuente para los gráficos
font_add_google("Poppins", "pop")
showtext_auto()

In [20]:
# Definir URL base y parte intermedia
url_basica <- "https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte <- "microdatos/enigh"

# Definir los años y los tipos de archivos a descargar
years <- c("2018", "2020", "2022","2024")
tipos <- c("ns_hogares", "ns_poblacion","ns_viviendas")  # Tipos de archivos

# Crear la carpeta de destino si no existe
if (!dir.exists("data")) {
  dir.create("data")
}

# Descargar, descomprimir y renombrar los archivos
for (year in years) {
  for (tipo in tipos) {
    # Construir URL y nombres de archivo
    url <- paste0(url_basica, year, "/", segunda_parte, year, "_", tipo, "_csv.zip")
    zip_destfile <- paste0("data/", year, "_", tipo, ".zip")

    # Descargar solo si el archivo no existe
    if (!file.exists(zip_destfile)) {
      download.file(url, destfile = zip_destfile, mode = "wb")
    } else {
      message("Archivo ya descargado: ", zip_destfile)
    }

    # Extraer archivos en una subcarpeta específica
    extract_dir <- paste0("data/", year, "_", tipo)
    if (!dir.exists(extract_dir)) {
      dir.create(extract_dir)
    }

    unzip(zip_destfile, exdir = extract_dir)

    # Buscar el archivo CSV dentro de la carpeta extraída
    extracted_files <- list.files(extract_dir, pattern = "\\.csv$", full.names = TRUE)

    if (length(extracted_files) == 1) {
      # Determinar el nuevo nombre con formato adecuado
      new_name <- paste0("data/", tipo, "_", year, ".csv")

      # Mover el archivo al directorio principal y renombrarlo
      file.rename(extracted_files, new_name)

      message("Archivo renombrado y movido: ", new_name)
    } else {
      message("Error: No se encontró un archivo CSV único en ", extract_dir)
    }
  }
}


Archivo ya descargado: data/2018_ns_hogares.zip

Archivo renombrado y movido: data/ns_hogares_2018.csv

Archivo ya descargado: data/2018_ns_poblacion.zip

Archivo renombrado y movido: data/ns_poblacion_2018.csv

Archivo ya descargado: data/2018_ns_viviendas.zip

Archivo renombrado y movido: data/ns_viviendas_2018.csv

Archivo ya descargado: data/2020_ns_hogares.zip

Archivo renombrado y movido: data/ns_hogares_2020.csv

Archivo ya descargado: data/2020_ns_poblacion.zip

Archivo renombrado y movido: data/ns_poblacion_2020.csv

Archivo ya descargado: data/2020_ns_viviendas.zip

Archivo renombrado y movido: data/ns_viviendas_2020.csv

Archivo ya descargado: data/2022_ns_hogares.zip

Archivo renombrado y movido: data/ns_hogares_2022.csv

Archivo ya descargado: data/2022_ns_poblacion.zip

Archivo renombrado y movido: data/ns_poblacion_2022.csv

Archivo ya descargado: data/2022_ns_viviendas.zip

Archivo renombrado y movido: data/ns_viviendas_2022.csv

Archivo renombrado y movido: data/ns_hog

In [ ]:
estima_pobreza_telecom <- function(year, variable) {

  # --- HOGARES: leer y parsear SOLO con as.numeric ---
  hogares <- read_csv(paste0("data/ns_hogares_", year, ".csv"),
                      show_col_types = FALSE) %>%
    mutate(
      telefono   = as.numeric(telefono),
      celular    = as.numeric(celular),
      conex_inte = as.numeric(conex_inte),
      folioviv = as.character(folioviv),
      foliohog = as.character(foliohog)
    ) %>%
    # Indicadores telecom
    mutate(
      # Adecuación telefonía (fija o celular)
      ATlj = case_when(
        telefono == 1 & celular == 1 ~ 1.5,
        telefono == 1 | celular == 1 ~ 1.0,
        telefono == 2 & celular == 2 ~ 0.0,
        TRUE ~ NA_real_
      ),
      # Internet
      Ij  = case_when(
        conex_inte == 1 ~ 1,
        conex_inte == 2 ~ 0,
        TRUE ~ NA_real_
      ),
      IVj   = 1 - Ij,             # Carencia de internet
      ACTI  = (ATlj + Ij) / 2,    # Adecuación conjunta telefonía + internet
      CTELJ = 1 - ACTI            # Carencia conjunta
    ) %>%
    # Estratos de carencia 
    mutate(
      E_CTELJ = case_when(
        CTELJ <= -0.5                 ~ 6,
        CTELJ >  -0.5 & CTELJ <= -0.1 ~ 5,
        CTELJ >  -0.1 & CTELJ <=  0   ~ 4,
        CTELJ >   0   & CTELJ <=  1/3 ~ 3,
        CTELJ >   1/3 & CTELJ <=  0.5 ~ 2,
        CTELJ >   0.5 & CTELJ <=  1   ~ 1,
        TRUE ~ NA_real_
      ),
      pobreza_ctelj = case_when(   # Pobreza por carencia de telecom
        E_CTELJ >= 4 ~ 0,
        E_CTELJ <= 3 ~ 1,
        TRUE ~ NA_real_
      )
    )

  # --- Traer variables de diseño según año ---
  if (year <= 2020) {
    # Leer VIVIENDAS solo para traer upm, est_dis, factor
    viviendas_diseno <- read_csv(paste0("data/ns_viviendas_", year, ".csv"),
                                 show_col_types = FALSE) %>%
      select(folioviv, upm, est_dis, factor)

    hogares <- hogares %>%
      left_join(viviendas_diseno %>% mutate(folioviv = as.character(folioviv)),
                by = "folioviv") %>%
      select(folioviv, foliohog, ATlj, Ij, IVj, ACTI, CTELJ, E_CTELJ, pobreza_ctelj,
             upm, est_dis, factor)

  } else if (year >= 2022) {
    hogares <- hogares %>%
      select(folioviv, foliohog, ATlj, Ij, IVj, ACTI, CTELJ, E_CTELJ, pobreza_ctelj)
  }

  # --- Población: leer, limpiar núcleo y crear cve_ent ---
  pob <- read_csv(paste0("data/ns_poblacion_", year, ".csv"),
                  show_col_types = FALSE) %>%
    janitor::clean_names() %>%
    filter(!(parentesco >= 400 & parentesco < 500),
           !(parentesco >= 700 & parentesco < 800)) %>%
    mutate(
      cve_ent = case_when(
        nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
        nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
      ),
      folioviv = as.character(folioviv),
      foliohog  = as.character(foliohog)
    ) %>%
    left_join(hogares, by = c("folioviv","foliohog"))

  # --- Diseño muestral ---
  diseno <- pob %>%
    as_survey_design(
      ids     = upm,
      strata  = est_dis,
      weights = factor,
      nest    = TRUE
    )

  # --- Estimación (CDMX = "09") ---
  if (variable == "pobreza_telecom") {
    out <- diseno %>%
      filter(cve_ent == "09") %>%
      group_by(pobreza_ctelj) %>%
      summarise(
        pob        = survey_total(vartype = "cv"),
        porcentaje = survey_prop(vartype = "cv")
      ) %>%
      mutate(
        porcentaje = porcentaje * 100,
        year = year
      ) %>%
      filter(pobreza_ctelj == 1)
    return(out)
  } else {
    stop("Variable no reconocida")
  }
}


In [22]:
tabla_pobreza_telecom <- map_dfr(years, estima_pobreza_telecom, variable = "pobreza_telecom")
tabla_pobreza_telecom

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or mo

pobreza_ctelj,pob,pob_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,3214292,0.04696462,34.95216,0.04224707,2018
1,2071178,0.05309816,22.41378,0.05139124,2020
1,1835975,0.05605723,19.73210,0.05379468,2022
1,1154729,0.07876607,12.35590,0.07716588,2024


In [23]:
tabla_pobreza_telecom%>%
  select(-pobreza_ctelj) %>%
  rename(
    Año           = year,
    Personas      = pob,
    Personas_cv   = pob_cv,
    Porcentaje    = porcentaje,
    Porcentaje_cv = porcentaje_cv
  ) %>%
  relocate(Año, .before = Personas) %>%
  gt() %>%
  # textos de la segunda fila de encabezados
  cols_label(
    Personas      = "Valor",
    Personas_cv   = "Coeficiente de variación",
    Porcentaje    = "Valor",
    Porcentaje_cv = "Coeficiente de variación"
  ) %>%
  tab_spanner("Personas",   id = "sp_personas",   columns = c(Personas, Personas_cv)) %>%
  tab_spanner("Porcentaje", id = "sp_porcentaje", columns = c(Porcentaje, Porcentaje_cv)) %>%

  # --- Poppins + centrado ---
  opt_table_font(font = list(google_font("Poppins"), default_fonts())) %>%   # fuente
  tab_options(
    heading.align = "center",
    table.align   = "center",
    table.font.size = px(14),
    table.background.color = "white",
    table.border.top.color = "white",
    table.border.bottom.color = "white",
    data_row.padding = px(3),
    column_labels.font.weight = "bold",
    column_labels.border.top.color = "black",
    column_labels.border.bottom.color = "black",
    column_labels.border.bottom.width = px(2),
    table_body.hlines.color = "lightgrey",
    table_body.hlines.width = px(0.5),
    table_body.border.bottom.color = "black",
    table_body.border.bottom.width = px(2),
    source_notes.font.size = px(10),
    source_notes.padding = px(1)
  ) %>%
  # centrar celdas, labels y spanners
  cols_align(align = "center", columns = everything()) %>%
  tab_style(
    style = cell_text(align = "center"),
    locations = list(
      cells_column_labels(columns = everything()),
      cells_column_spanners(spanners = everything())
    )
  ) %>%

  fmt_number(columns = c(Personas, Personas_cv, Porcentaje, Porcentaje_cv), decimals = 2) %>%
  tab_header(
    title = md("**Pobreza de telecomunicaciones en CDMX**"),
    subtitle = md("**Personas en pobreza de telecomunicaciones (%)**")
  ) %>%
  tab_source_note(md("Nota: Resultados preliminares de acuerdo con la metodología de Evalúa CDMX")) %>%
  tab_source_note(md("Fuente: @claudiodanielpc con información de INEGI. Encuesta Nacional de Ingresos y Gastos de los Hogares (ENIGH)")) %>%
  gtsave("pobreza_telecomunicaciones.png")